<a href="https://colab.research.google.com/github/aldabbagh/projects/blob/master/Latent_space_walking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Minimal KerasCV notebook for latent space walking via prompt interpolation

Let's start by installing what we need.

In [1]:
!pip install tensorflow keras_cv --upgrade --quiet
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

     |████████████████████████████████| 578.0 MB 18 kB/s 
     |████████████████████████████████| 394 kB 66.2 MB/s 
     |████████████████████████████████| 438 kB 65.5 MB/s 
     |████████████████████████████████| 5.9 MB 59.3 MB/s 
     |████████████████████████████████| 1.7 MB 60.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 2 not upgraded.
Need to get 430 MB of archives.
After this operation, 1,392 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 6s (66

In [2]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from PIL import Image

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(img_height=512, img_width=512, jit_compile=True)

1356917/1356917 [==============================] - 0s 0us/step
By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
198180272/198180272 [==============================] - 3s 0us/step


Here's a gif saving utility that will come in handy.

In [ ]:
def export_as_gif(filename, images, frames_per_second=13, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )

## These are our configurable parameters

We will walk from `prompt1` to `prompt2`.

In [ ]:
prompt_1 = "Mysterious ruins in the snow, concept art, digital, artstation"
prompt_2 = "Mysterious ruins in the summer sunlight, concept art, digital, artstation"
interpolation_steps = 50
filename = "ruins_1.gif"

## Now let's generate

In [ ]:
encoding_1 = tf.squeeze(model.encode_text(prompt_1))
encoding_2 = tf.squeeze(model.encode_text(prompt_2))

batch_size = 10 # This value might to be lowered to 3 on a smaller GPU.
batches = interpolation_steps // batch_size

interpolated_encodings = tf.linspace(encoding_1, encoding_2, interpolation_steps)
batched_encodings = tf.split(interpolated_encodings, batches)

seed = 1337
noise = tf.random.normal((512 // 8, 512 // 8, 4), seed=seed)

images = []
for batch in range(batches):
    images += [
        Image.fromarray(img)
        for img in model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=25,
            diffusion_noise=noise,
        )
    ]

export_as_gif(filename, images, rubber_band=True)

25/25 [==============================] - 8s 339ms/step


To see the generated gif, look for the file browser tab on your left.